<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2020-10-12 16:14:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  15.9MB/s    in 40s     

2020-10-12 16:15:17 (16.1 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [2]:
import math
import numpy as np

from tqdm import tqdm
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader
import zipfile

import seaborn as sns


In [3]:
!wget https://raw.githubusercontent.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/master/Week%203/data.py

--2020-10-12 16:15:20--  https://raw.githubusercontent.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/master/Week%203/data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10563 (10K) [text/plain]
Saving to: ‘data.py’

data.py             100%[===================>]  10.32K  --.-KB/s    in 0.001s  

2020-10-12 16:15:20 (10.2 MB/s) - ‘data.py’ saved [10563/10563]



In [4]:
!mv data.py mydata.py

In [5]:
from mydata import Downloader, Parser

In [6]:
data_path = './data/'

In [7]:
downloader = Downloader(data_path=data_path)

In [8]:
downloader.run()

multiple: 100%|██████████| 17/17 [00:35<00:00,  2.07s/it]


In [9]:
parser = Parser(data_path=data_path)

In [10]:
unlabeled, train, valid = parser.run()

Reading: 100%|██████████| 38/38 [02:36<00:00,  4.12s/it]


In [11]:
unlabeled

,question,response
0,how man inches round is this?,"it is 6"" same as amazon's description!"
1,"i was reading for a rotary vane pump, to not s...",if you have an inline valve to lock the vacuum...
2,does it come with the screen protector like th...,yes it does
3,does this product work for older women?,"i am ""mature""! and i like it. hopefully you too"
4,why does the four port cost more than the five...,both the white and black color version of this...
...,...,...
137275,will these fit a 2002 toyota celica gts?,"it may take a little stretching, as it did my ..."
137276,does this package come with the charger for th...,thank you for your question. the baby touch bo...
137277,is this set the new angled bottle?,it has 2 ways to keep it straight and angled
137278,how much water does it hold? do you have to em...,"it's recommended to hold 19 liters, though the..."


In [12]:
train

,question,response,category
0,does it include a built-in pump?,"no, mine did not come with a built in pump. it...",sports and outdoors
1,does this bed plug in,this is a foam bed not an inflatable air bed.,pet supplies
2,are these mirrors power and will they work wit...,"no power,no signal, these mirrors are model sp...",automotive
3,will the water freeze in below freezing temps?,i had the same question as my dogs stay outsid...,pet supplies
4,does it have battery backup for power outages?,no battery backup,office products
...,...,...,...
249995,do these come with bearings?,"just wheels, no bearings.",sports and outdoors
249996,does this work on a 2000chevy s-10 4.3,we can confirm the parts application with the ...,automotive
249997,can i use it for blooming gardenia? have bad s...,have quite a lot of experience with this produ...,sports and outdoors
249998,will this fit a car with a short spoiler? its ...,i'm also curious whether this bike rack will f...,sports and outdoors


In [13]:
valid

,question,response,category
0,inform the size,regular size,automotive
1,how does it light up?,it does look like in the picture. if you take ...,sports and outdoors
2,can the sneaky pete initials be removed withou...,the holster that i have is embossed. so the s...,sports and outdoors
3,for example: when this device is charging my 5...,"i don't see why not, but it may slow your char...",cell phones and accessories
4,does this bleeder come with a resivour cap tha...,very good cap fits all bmw's that i know of. i...,automotive
...,...,...,...
49995,do you ship to the u k please?,"yes, when making an international order we fir...",sports and outdoors
49996,what size are the socks?,"i think it was labeled 0-6 month, but we felt ...",baby
49997,can you take off the quad rail on the pt85,"yes, and you can take off the composer but the...",sports and outdoors
49998,are silicone bristles enough to get baby's tee...,i think it looks cute and soft and good for ba...,baby


In [14]:
# проверим, что в трейне и валидации одинаковые категории
set(train.category.unique().tolist()) == set(valid.category.unique().tolist())

True

In [15]:
train.category.unique()

array(['sports and outdoors', 'pet supplies', 'automotive',
       'office products', 'cell phones and accessories', 'beauty', 'baby',
       'grocery and gourmet food'], dtype=object)

In [16]:
unique_categories = set(train.category.unique().tolist() + valid.category.unique().tolist())

In [17]:
category2index = {category: index for index, category in enumerate(unique_categories)}

In [18]:
category2index

{'automotive': 2,
 'baby': 0,
 'beauty': 5,
 'cell phones and accessories': 6,
 'grocery and gourmet food': 7,
 'office products': 4,
 'pet supplies': 3,
 'sports and outdoors': 1}

In [19]:
train['target'] = train.category.map(category2index)
valid['target'] = valid.category.map(category2index)

In [20]:
from torch.utils.data import Dataset, DataLoader

In [21]:
# игрушечный датасет
# 121535 примера, 4 фичи, 3 класса
some_data_x = np.random.rand(121535, 4)
some_data_y = np.random.randint(3, size=(121535,))

In [22]:

# просто рандомные цифры
some_data_x[:10]

array([[0.14281792, 0.96065793, 0.35854546, 0.46609858],
       [0.78829209, 0.99061179, 0.93469212, 0.63162634],
       [0.56110753, 0.32590169, 0.29516193, 0.66546604],
       [0.05460179, 0.56229444, 0.25854876, 0.74157347],
       [0.78322014, 0.71402004, 0.82926823, 0.02103047],
       [0.03088728, 0.16838446, 0.05238811, 0.77739535],
       [0.92362605, 0.58298071, 0.96170413, 0.04724381],
       [0.1156664 , 0.94897289, 0.72421643, 0.77475617],
       [0.1441345 , 0.66750075, 0.14935046, 0.55240946],
       [0.13499113, 0.04023578, 0.97030316, 0.74390511]])

In [23]:
some_data_y

array([2, 1, 2, ..., 2, 0, 2])

In [24]:
batch_size = 16

for i_batch in range(math.ceil(some_data_x.shape[0] / batch_size)):
    
    x_batch = some_data_x[i_batch * batch_size:(i_batch + 1) * batch_size]
    y_batch = some_data_y[i_batch * batch_size:(i_batch + 1) * batch_size]
    
    x_batch = torch.tensor(x_batch)
    y_batch = torch.tensor(y_batch)
    
    break

In [25]:
x_batch

tensor([[0.1428, 0.9607, 0.3585, 0.4661],
        [0.7883, 0.9906, 0.9347, 0.6316],
        [0.5611, 0.3259, 0.2952, 0.6655],
        [0.0546, 0.5623, 0.2585, 0.7416],
        [0.7832, 0.7140, 0.8293, 0.0210],
        [0.0309, 0.1684, 0.0524, 0.7774],
        [0.9236, 0.5830, 0.9617, 0.0472],
        [0.1157, 0.9490, 0.7242, 0.7748],
        [0.1441, 0.6675, 0.1494, 0.5524],
        [0.1350, 0.0402, 0.9703, 0.7439],
        [0.4586, 0.4394, 0.7310, 0.3827],
        [0.3263, 0.9888, 0.2229, 0.6774],
        [0.8750, 0.1231, 0.9995, 0.9140],
        [0.4950, 0.2194, 0.7018, 0.8395],
        [0.2697, 0.0832, 0.7929, 0.4384],
        [0.4111, 0.6586, 0.7293, 0.5973]], dtype=torch.float64)

In [26]:
x_batch.shape, y_batch.shape

(torch.Size([16, 4]), torch.Size([16]))

In [82]:
class ToyDataset(Dataset):
    
    def __init__(self, data_x, data_y):
        
        super().__init__()
        
        self.data_x = data_x
        self.data_y = data_y
        
    def __len__(self):
        
        # нужно обязательно определить эту функцию
        # должна возвращать размер датасета
        # нужен для DataLoader, чтобы семплировать батчи
        
        return len(self.data_x)
    
    @staticmethod
    def pow_features(x, n=2):
        
        return x ** n
    
    @staticmethod
    def log_features(x):
        
        return np.log(x)
    
    def __getitem__(self, idx):
        
        # еще нужно определить этот метод
        # то есть как мы будем доставать наши данные по индексу
        
        x = self.data_x[idx]
        
        # внутри датасета мы можем делать все что угодно с нашими данными
        # например выше определим функции, которые добавляют степенные фичи
        x_p_2 = self.pow_features(x, n=2)
        x_p_3 = self.pow_features(x, n=3)
        # и еще возьмем логарифмические фичи
        x_log = self.log_features(x)
        
        # сконкатенируем наши фичи
        x = np.concatenate([x, x_p_2, x_p_3, x_log])
        
        y = self.data_y[idx]
        
        return x, y

In [83]:
toy_dataset = ToyDataset(some_data_x, some_data_y)

In [84]:
toy_loader = DataLoader(dataset=toy_dataset, batch_size=128)

In [85]:
for x, y in toy_loader:
    break

In [87]:

# сделаем небольшую модель и посчитаем лосс

model = torch.nn.Sequential(torch.nn.Linear(16, 8),
                            torch.nn.ReLU(),
                            torch.nn.Linear(8, 4),
                            torch.nn.ReLU(),
                            torch.nn.Linear(4, 3))

criterion = torch.nn.CrossEntropyLoss()

with torch.no_grad():

    prediction = model(x.float())

    loss = criterion(prediction, y)
    
loss.item()

1.0921432971954346

In [88]:
class TextClassificationDataset(Dataset):
    
    def __init__(self, texts, targets):
        super().__init__()
        
        self.texts = texts
        self.targets = targets
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        
        text = self.texts[index]
        target = self.targets[index]
        
        return text, target

In [89]:

# подготовим данные
train_x = list(train.question)
train_y = list(train.target)

valid_x = list(valid.question)
valid_y = list(valid.target)

In [90]:
train_dataset = TextClassificationDataset(texts=list(train.question), targets=list(train.target))

In [91]:
text, target = train_dataset[0]

In [92]:
def load_embeddings(zip_path, filename, pad_token='PAD', max_words=100_000, verbose=True):
    
    vocab = dict()
    embeddings = list()

    with zipfile.ZipFile(zip_path) as zipped_file:
        with zipped_file.open(filename) as file_object:

            vocab_size, embedding_dim = file_object.readline().decode('utf-8').strip().split()

            vocab_size = int(vocab_size)
            embedding_dim = int(embedding_dim)
            
            # в файле 1 000 000 слов с векторами, давайте ограничим для простоты этот словарь
            max_words = vocab_size if max_words <= 0 else max_words
            
            # добавим пад токен и эмбеддинг в нашу матрицу эмбеддингов и словарь
            vocab[pad_token] = len(vocab)
            embeddings.append(np.zeros(embedding_dim))

            progress_bar = tqdm(total=max_words, disable=not verbose)

            for line in file_object:
                parts = line.decode('utf-8').strip().split()

                token = ' '.join(parts[:-embedding_dim]).lower()
                
                if token in vocab:
                    continue
                
                word_vector = np.array(list(map(float, parts[-embedding_dim:])))

                vocab[token] = len(vocab)
                embeddings.append(word_vector)

                progress_bar.update()
                
                if len(vocab) == max_words:
                    break

            progress_bar.close()

    embeddings = np.stack(embeddings)
    
    return vocab, embeddings

In [93]:
vocab, embeddings = load_embeddings('./wiki-news-300d-1M.vec.zip', 'wiki-news-300d-1M.vec', max_words=100_000)

100%|█████████▉| 99999/100000 [00:12<00:00, 8278.11it/s]


In [94]:
index2token = {index: token for token, index in vocab.items()}

In [95]:
emb_norms = np.linalg.norm(embeddings, axis=1)

In [96]:
def get_k_nearest_neighbors(word, embeddings, emb_norms, vocab, index2token, k=5):
    
    if word not in vocab:
        print('Not in vocab')
        return
    
    word_index = vocab[word]

    word_vector = embeddings[word_index]
    word_vector = np.expand_dims(word_vector, 0)

    scores = (word_vector @ embeddings.T)[0]
    
    # переводим в косинусы, поделив на нормы векторов
    # эпсилон 1e-6 для того, чтобы не делить на 0
    scores = scores / (emb_norms + 1e-6) / emb_norms[word_index]
    
    # 1:k+1 потому что первый вариант это само слово
    for idx in scores.argsort()[::-1][1:k+1]:
        print(f'Слово {index2token[idx]} близко на {scores[idx]:.2f} к слову {word}')

In [97]:
total_n_words = 0
unknown_words = list()

for sample in tqdm(train_x):
    # токенизация
    tokens = wordpunct_tokenize(sample)
    
    for tok in tokens:
        # проверяем есть ли токен в нашем словаре
        if tok not in vocab:
            unknown_words.append(tok)
            
        total_n_words += 1
        
print(f'Мы не знаем {len(unknown_words)} слов из {total_n_words} слов в датасете')
print(f'Что составляет {len(unknown_words) * 100 / total_n_words:.2f}% датасета')
print()
print(f'Уникальных неизвестных слов: {len(set(unknown_words))}')

100%|██████████| 250000/250000 [00:03<00:00, 82472.27it/s] 

Мы не знаем 110907 слов из 4203856 слов в датасете
Что составляет 2.64% датасета

Уникальных неизвестных слов: 36053


In [98]:
class TextClassificationDataset(Dataset):
    
    def __init__(self, texts, targets, vocab):
        super().__init__()
        
        self.texts = texts
        self.targets = targets
        self.vocab = vocab
        
    def __len__(self):
        return len(self.texts)
    
    def tokenization(self, text):
        
        tokens = wordpunct_tokenize(text)
        
        token_indices = [self.vocab[tok] for tok in tokens if tok in self.vocab]
        
        return token_indices
    
    def __getitem__(self, index):
        
        text = self.texts[index]        
        target = self.targets[index]
        
        tokenized_text = self.tokenization(text)
        
        # переведем наши индексы токенов в торчовый тензор
        # таргет переведется самостоятельно
        tokenized_text = torch.tensor(tokenized_text)
        
        return tokenized_text, target

In [99]:
train_dataset = TextClassificationDataset(texts=train_x, targets=train_y, vocab=vocab)

In [100]:
x, y = train_dataset[5]

In [102]:
# [index2token[idx.item()] for idx in x]

In [103]:
train_lengths = [len(wordpunct_tokenize(sample)) for sample in tqdm(train_x)]

100%|██████████| 250000/250000 [00:01<00:00, 193995.94it/s]


In [104]:
class TextClassificationDataset(Dataset):
    
    def __init__(self, texts, targets, vocab, pad_index=0, max_length=32):
        super().__init__()
        
        self.texts = texts
        self.targets = targets
        self.vocab = vocab
        
        self.pad_index = pad_index
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def tokenization(self, text):
        
        tokens = wordpunct_tokenize(text)
        
        token_indices = [self.vocab[tok] for tok in tokens if tok in self.vocab]
        
        return token_indices
    
    def padding(self, tokenized_text):
        
        tokenized_text = tokenized_text[:self.max_length]
        
        tokenized_text += [self.pad_index] * (self.max_length - len(tokenized_text))
        
        return tokenized_text
    
    def __getitem__(self, index):
        
        text = self.texts[index]        
        target = self.targets[index]
        
        tokenized_text = self.tokenization(text)
        tokenized_text = self.padding(tokenized_text)
        
        tokenized_text = torch.tensor(tokenized_text)
        
        return tokenized_text, target

In [105]:
train_dataset = TextClassificationDataset(texts=train_x, targets=train_y, vocab=vocab)

In [106]:
x, y = train_dataset[0]

In [108]:
train_dataset = TextClassificationDataset(texts=train_x, targets=train_y, vocab=vocab)
valid_dataset = TextClassificationDataset(texts=valid_x, targets=valid_y, vocab=vocab)

train_loader = DataLoader(train_dataset, batch_size=128)
valid_loader = DataLoader(valid_dataset, batch_size=128)

In [124]:

for x, y in train_loader:
    print(x, y)
    break

tensor([[ 218,   20,  264,  ...,    0,    0,    0],
        [ 218,   29, 2591,  ...,    0,    0,    0],
        [  32,  130, 7624,  ...,    0,    0,    0],
        ...,
        [  28,  231,    6,  ...,    0,    0,    0],
        [  29, 1607, 1101,  ...,    0,    0,    0],
        [  97,   30,  231,  ...,  168,    5,  730]]) tensor([1, 3, 2, 3, 4, 1, 6, 1, 6, 1, 2, 6, 3, 5, 6, 2, 1, 2, 2, 0, 4, 2, 5, 1,
        4, 6, 7, 1, 3, 2, 1, 6, 2, 1, 0, 6, 6, 7, 2, 2, 0, 5, 5, 3, 6, 3, 2, 1,
        1, 1, 1, 2, 1, 3, 1, 1, 2, 4, 4, 6, 4, 2, 1, 6, 5, 4, 6, 6, 4, 0, 2, 0,
        5, 2, 2, 3, 2, 2, 1, 1, 5, 1, 6, 5, 6, 1, 5, 6, 4, 3, 3, 2, 2, 0, 5, 6,
        3, 3, 4, 4, 1, 1, 6, 2, 0, 3, 4, 3, 2, 1, 1, 2, 5, 4, 1, 5, 6, 4, 2, 1,
        2, 2, 1, 3, 5, 1, 6, 5])


In [123]:
list(train_loader)[0]

[tensor([[ 218,   20,  264,  ...,    0,    0,    0],
         [ 218,   29, 2591,  ...,    0,    0,    0],
         [  32,  130, 7624,  ...,    0,    0,    0],
         ...,
         [  28,  231,    6,  ...,    0,    0,    0],
         [  29, 1607, 1101,  ...,    0,    0,    0],
         [  97,   30,  231,  ...,  168,    5,  730]]),
 tensor([1, 3, 2, 3, 4, 1, 6, 1, 6, 1, 2, 6, 3, 5, 6, 2, 1, 2, 2, 0, 4, 2, 5, 1,
         4, 6, 7, 1, 3, 2, 1, 6, 2, 1, 0, 6, 6, 7, 2, 2, 0, 5, 5, 3, 6, 3, 2, 1,
         1, 1, 1, 2, 1, 3, 1, 1, 2, 4, 4, 6, 4, 2, 1, 6, 5, 4, 6, 6, 4, 0, 2, 0,
         5, 2, 2, 3, 2, 2, 1, 1, 5, 1, 6, 5, 6, 1, 5, 6, 4, 3, 3, 2, 2, 0, 5, 6,
         3, 3, 4, 4, 1, 1, 6, 2, 0, 3, 4, 3, 2, 1, 1, 2, 5, 4, 1, 5, 6, 4, 2, 1,
         2, 2, 1, 3, 5, 1, 6, 5])]

In [117]:
embedding_layer = nn.Embedding(num_embeddings=len(vocab), 
                               embedding_dim=embeddings.shape[-1],
                               padding_idx=0)

In [125]:
embeddings = torch.tensor(embeddings).float()
embedding_layer = nn.Embedding.from_pretrained(embeddings, padding_idx=0)
x_embed = embedding_layer(x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [126]:
lstm = nn.LSTM(input_size=300, hidden_size=128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)

In [127]:
x_lstm, _ = lstm(x_embed)

In [128]:
x_lstm.mean(dim=1).shape

torch.Size([128, 256])

In [129]:
class DeepAverageNetwork(nn.Module):
    
    def __init__(self, embeddings, linear_1_size, linear_2_size, n_classes):
        super().__init__()
        
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings, padding_idx=0)
        
        self.batch_norm = nn.BatchNorm1d(num_features=embeddings.shape[-1])
        
        self.linear_1 = nn.Linear(in_features=embeddings.shape[-1], out_features=linear_1_size)
        self.linear_2 = nn.Linear(in_features=linear_1_size, out_features=linear_2_size)
        self.linear_3 = nn.Linear(in_features=linear_2_size, out_features=n_classes)
        
    def forward(self, x):
        
        # переводим индексы слов в эмбеддинги этих слов
        # (batch_size, sequence_length) -> (batch_size, sequence_length, embedding_dim)
        x = self.embedding_layer(x)
        
        # агрегируем наши эмбеддинги по размерности время
        # (batch_size, sequence_length, embedding_dim) -> (batch_size, embedding_dim)
        x = x.sum(dim=1)
        
        # делаем нормирование
        # (batch_size, embedding_dim) -> (batch_size, embedding_dim)
        x = self.batch_norm(x)
        
        # прогоняем через первый линейный слой
        # (batch_size, embedding_dim) -> (batch_size, linear_1_size)
        x = self.linear_1(x)
        
        # применяем нелинейность
        # (batch_size, linear_1_size) -> (batch_size, linear_1_size)
        x = torch.relu(x)
        
        # прогоняем через второй линейный слой
        # (batch_size, linear_1_size) -> (batch_size, linear_2_size)
        x = self.linear_2(x)
        
        # применяем нелинейность
        # (batch_size, linear_2_size) -> (batch_size, linear_2_size)
        x = torch.relu(x)
        
        # переводим с помощью линейного преобразования в количество классов
        # (batch_size, linear_2_size) -> (batch_size, n_classes)
        x = self.linear_3(x)
        
        ## по идеи здесь должен был быть софтмакс
        ## но мы будем использовать лосс nn.CrossEntropyLoss()
        ## в его документации написано
        ## This criterion combines :func:`nn.LogSoftmax` and :func:`nn.NLLLoss` in one single class.
        ## это некоторая оптимизация, которая включает в себя сразу и софтмакс и сам negative log likelihood лосс
        ## так как у нас в лоссе есть софтмакс, то мы не будем применять его в сетке
        ## на этапе предсказания (а не обучения) мы будем отдельно делать софтмакс для получения распределения классов
        ## 
        ## (batch_size, n_classes) -> (batch_size, n_classes)
        # x = torch.softmax(x, dim=-1)
        
        return x

In [130]:
model = DeepAverageNetwork(embeddings=embeddings,
                           linear_1_size=256, 
                           linear_2_size=128, 
                           n_classes=len(category2index))

In [131]:
criterion = nn.CrossEntropyLoss()